In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K

Using TensorFlow backend.


In [2]:
gc.enable()

In [3]:
steps = 20
height = 108
width = 108

In [4]:
def main(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [5]:
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    wrong = 0
    for i in range(prediction.shape[0]):
        for j in range(prediction.shape[1]):
            if abs(prediction[i][j] - y_test[i][j]) > 0.5:
                wrong += 1
                break
    print('{} samples are loaded for testing'.format(prediction.shape[0]))
    print('{} testing samples are predicted wrong'.format(wrong))
    accuracy = 1 - wrong/(prediction.shape[0])
    return accuracy

In [6]:
classifier = main(optimizer='adam', loss='binary_crossentropy')
classifier.load_weights('weight-75-overlap-model2.hdf5')

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


In [7]:
#This is an example

In [8]:
#change the path to where you save the input numpy arrays
path = 'E:/notes_database/75_overlap/'
lstm = np.array(glob.glob(path+'lstm_inputs/*'))
cnn = np.array(glob.glob(path+'cnn_inputs/*'))
y_train = np.array(glob.glob(path+'y_train/*'))
c_lstm = np.array(glob.glob(path+'counter_lstm_inputs/*'))
c_cnn = np.array(glob.glob(path+'counter_cnn_inputs/*'))
c_y_train = np.array(glob.glob(path+'counter_y_train/*'))

In [9]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [10]:
def generate_test_arrays(test_order, num=0):
    cnn_inputs = np.load(cnn[test_order[num][0]])
    labels_inputs = np.load(y_train[test_order[num][0]])
    lstm_inputs = np.load(lstm[test_order[num][0]])
    
    lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[test_order[num][0]]), axis=0)
    cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[test_order[num][0]]), axis=0)
    labels_inputs = np.append(labels_inputs, np.load(c_y_train[test_order[num][0]]), axis=0)
    
    for file in test_order[num][1:]:
        
        lstm_inputs = np.append(lstm_inputs, np.load(lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(y_train[file]), axis=0)
        lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(c_y_train[file]), axis=0)
    
    return cnn_inputs, lstm_inputs, labels_inputs

In [11]:
a,b,c = generate_test_arrays(test_order)

In [12]:
accuracy = load_test_score(classifier, a, b, c)

73497 samples are loaded for testing
2568 testing samples are predicted wrong


In [13]:
accuracy

0.9650597983591167

In [14]:
# If you wanna see the output in the standard format
def return_prediction(prediction):
    output = 'The notes are: C4'
    for i in range(prediction.shape[0]):
        for j in range(prediction.shape[1]):
            if prediction[i][j] < 0.5:
                prediction[i][j] = 0
            elif prediction[i][j] > 0.5:
                prediction[i][j] = 1
                output+='_{}'.format(j+60)
        print(output)
    return prediction